In [17]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.document_loaders import DirectoryLoader,TextLoader

In [7]:
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

os.environ["GROQ_API_KEY"]=groq_api_key

In [4]:
llm=ChatGroq(
    model_name="llama-3.3-70b-versatile",
    api_key=groq_api_key,
)


AIMessage(content="Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How about you? How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 41, 'total_tokens': 94, 'completion_time': 0.116392799, 'prompt_time': 0.0020401, 'queue_time': 0.052654596, 'total_time': 0.118432899}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_34d416ee39', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--127549db-9837-41fa-ac1e-f40d77068a27-0', usage_metadata={'input_tokens': 41, 'output_tokens': 53, 'total_tokens': 94})

In [18]:

directoryLoader = DirectoryLoader(
    path="../data/practical_rag_data/",
    loader_cls=TextLoader,
    show_progress=True,
    glob="**/*.txt"
)

data=directoryLoader.load()
print(data[0])

text_splitter=RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,   
)

chunks=text_splitter.split_documents(data)
print(chunks[0],chunks[1])

100%|██████████| 4/4 [00:00<00:00, 1152.04it/s]

page_content='Climate Change
Climate change refers to long-term shifts in temperatures and weather patterns, largely driven by human activities such as burning fossil fuels. These activities increase greenhouse gas concentrations in the atmosphere, trapping heat and altering global climates. Consequences include rising sea levels, extreme weather events, and biodiversity loss.

Space Exploration
Humanity’s interest in space has evolved from simple stargazing to landing rovers on Mars. Organizations like NASA and private companies like SpaceX are pushing boundaries by exploring planets, planning moon bases, and even contemplating missions to Jupiter's moons. Space exploration drives technological innovation and inspires global collaboration.

Artificial Intelligence Ethics
As AI systems become more capable, ethical concerns arise about their usage, decision-making, and transparency. Issues like algorithmic bias, lack of accountability, and potential misuse in surveillance or warfare rai

In [19]:
contextualize_q_system_prompt = """Given a chat history and the latest user question 
which might reference context in the chat history, formulate a standalone question 
which can be understood without the chat history. Do NOT answer the question, 
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

In [21]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
hf = HuggingFaceEmbeddings(
    model_name=model_name,
)
# Creating Chroma DB

db=Chroma.from_documents(
    documents=chunks,
    embedding=hf,
    persist_directory="../rag_db",
    collection_name="rag_collection"
)

retriever=db.as_retriever(
    search_kwargs={"k":3}
)

In [ ]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# ----------------------------
# 1) Initialize LLM + Retriever
# ----------------------------

# Make sure you have your API key set:
# export OPENAI_API_KEY="sk-...."
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("Please set the OPENAI_API_KEY environment variable.")

# Chat model (adjust model name as you like)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create a tiny in-memory knowledge base
docs = [
    "Machine learning (ML) is a subfield of artificial intelligence focused on building systems that learn from data.",
    "Common types of machine learning include supervised learning, unsupervised learning, and reinforcement learning.",
    "Supervised learning uses labeled data to learn a mapping from inputs to outputs.",
    "Unsupervised learning finds patterns or structure in unlabeled data (e.g., clustering, dimensionality reduction).",
    "Reinforcement learning trains an agent to act in an environment by maximizing cumulative reward."
]

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_texts(docs, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# -------------------------------------
# 2) Build the history-aware retriever
# -------------------------------------

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

history_aware_retriever = create_history_aware_retriever(
    llm=llm,
    retriever=retriever,
    prompt=contextualize_q_prompt
)

# -------------------------------------
# 3) Build the QA chain (stuff strategy)
# -------------------------------------

qa_system_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Context: {context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

# -------------------------------------
# 4) Compose the full conversational RAG
# -------------------------------------

conversational_rag_chain = create_retrieval_chain(
    retriever=history_aware_retriever,
    combine_docs_chain=question_answer_chain
)

print("✅ Conversational RAG chain created!\n")

# -------------------------------------
# 5) Demo run
# -------------------------------------

chat_history = []

# First question
result1 = conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "What is machine learning?"
})

print("Q1: What is machine learning?")
print("A1:", result1.get("answer", result1))
print()

# Update chat history
chat_history.extend([
    HumanMessage(content="What is machine learning?"),
    AIMessage(content=result1.get("answer", "")),
])

# Follow-up question (history-aware)
result2 = conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "What are its main types?"
})

print("Q2: What are its main types?")
print("A2:", result2.get("answer", result2))
print()

# (Optional) Ask another follow-up to see reformulation in action
chat_history.extend([
    HumanMessage(content="What are its main types?"),
    AIMessage(content=result2.get("answer", "")),
])

result3 = conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "Give me one-sentence definitions for each."
})



## CONVERSATIONAL MEMORY USING RUNNABLES

In [ ]:
# pip install langchain langchain-core langchain-community langchain-openai faiss-cpu tiktoken

import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# --- base LLM & retriever ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
emb = OpenAIEmbeddings(model="text-embedding-3-small")
docs = [
    "Machine learning is a subfield of AI that learns patterns from data.",
    "The main types of ML are supervised, unsupervised, and reinforcement learning.",
]
vs = FAISS.from_texts(docs, emb)
retriever = vs.as_retriever(search_kwargs={"k": 4})

# --- contextualizer ---
contextualize_q_system = """Given chat history + latest user Q, rewrite it as a standalone question.
Don't answer; just return the rewritten question (or the original if already standalone)."""
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])
history_aware_retriever = create_history_aware_retriever(
    llm=llm, retriever=retriever, prompt=contextualize_q_prompt
)

# --- QA chain ---
qa_system = """You are an assistant for QA. Use retrieved context only.
If unknown, say you don't know. Max three sentences.

Context: {context}"""
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])
qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

# --- Conversational RAG ---
rag = create_retrieval_chain(history_aware_retriever, qa_chain)

# --- Add memory (no more manual chat_history handling) ---
store = {}  # session_id -> InMemoryChatMessageHistory

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

rag_with_memory = RunnableWithMessageHistory(
    rag,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",   # typical key produced by create_retrieval_chain
)

# --- Use it: just pass a session_id in config ---
cfg = {"configurable": {"session_id": "user-42"}}

print(rag_with_memory.invoke({"input": "What is machine learning?"}, cfg)["answer"])
print(rag_with_memory.invoke({"input": "What are its main types?"}, cfg)["answer"])
